# Measurements in systems with multiple qubits

In the previous tutorial, we discussed the concept of measurements done on single qubit systems. Building upon those ideas, this tutorial will introduce you to the concepts involved in, and the `Q#` implementation of measurements done on multi-qubit systems. This will involved measuring a single qubit in a multi-qubit system, as well as measuring multiple qubits simultaneously. 

We recommend to go through the [tutorial that introduces single qubit system measurements](../SingleQubitGates/SingleQubitGates.ipynb) before starting this one.

This tutorial covers the following topics for multiple qubit systems:

* Measuring qubits one at a time
* Measuring multiple qubits simultaneously
$\newcommand{\ket}[1]{\left|#1\right>}$
$\newcommand{\bra}[1]{\left<#1\right|}$

### Theoretical basics: Partial and complete measurements
Measurements in multi-qubit systems can involve measuring some of the qubits, or all of the qubits simultaneously. Somewhat counter-intuitively, complete measurements (i.e. measurements done on all the qubits simultaneously) are easier to understand than partial measurements. We start this tutorial with a discussion of measurements involving all the qubits in a system. As we will see in the section below, they are a straightforward generalization of measurements in single-qubit systems.

## Measurement probabilities when all qubits are measured simultaneously
Consider a system consisting of $n\geq1$ qubits. The wavefunction of such a system will belong to a vector space of dimension $2^n$. Consequently, any orthogonal basis for this space will consist of $2^n$ vectors. An example is the computational basis, consisting of vectors $|0\dotsc0\rangle, \dotsc, |1\dotsc 1\rangle$, where each ket contains a bitstring of length n. For generality, we consider an arbitrary orthonormal basis, which we denote by $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$. Then, the state $|\psi\rangle$ of the multi-qubit system can be expressed as a linear combination of the $2^n$ basis vectors. That is, there exist complex numbers $c_0,c_1,\dotsc, c_{2^n-1}$ such that
 $$
 |\psi\rangle = \sum_{i=0}^{2^n-1} c_i|b_i\rangle.
 $$
 In line with the usual convention, we choose the wavefunction to be normalized, so that $|c_0|^2 + \dotsc + |c_{2^n-1}|^2 =1$. Then, a quantum measurement in the $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$ basis satisfies the following rules:
 - The measurement outcome $b_i$ occurs with probability $|c_i|^2$. 
 - Whenever the measurement outcome is $b_i$, the wavefunction collapses to the state $|b_i\rangle$. That is, the post measurement state of the system is equal to $|b_i\rangle$.
 
> Similar to measurements in single qubit systems, the assumption of normalization of the original wavefunction is required for the applicability of the statements above, in order to ensure that the sum of all the outcome probabilities is one.

> Whenever the wavefunction of a system is known in a certain basis, but a measurement is implemented in another basis, to obtain the measurement probabilities, it is best to first express the wavefunction in the basis in which measurements are to be done, and then apply the formulas above. 

### <span style="color:blue">Exercise 1</span>: Multi-qubit measurement outcome probabilities I 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{1}{3}\ket {00} + \frac{2}{3} \ket {01} + \frac{2}{3}\ket {11}$$.

If all the qubits are measured simultaneously in the computational basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [--- Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


### <span style="color:blue">Exercise 2</span>: Multi-qubit measurement outcome probabilities II 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{2}{3}\ket {00} + \frac{1}{3} \ket {01} + \frac{2}{3}\ket {11}$$.

If all the qubits are measured simultaneously in the Pauli X basis, i.e. in the $\{ \ket{++}, \ket{+-}, \ket{-+}, \ket{--}\}$ basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [--- Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


### Measuring all qubits in a system one after another

Although it is possible to measure all qubits in a quantum mechanics system simultaneously, in practice, the qubits are measured one after another on a quantum computer. This distinction is not important for an ideal computer: in terms of the outcomes and measurement probabilities, measuring all the qubits in a system one after another is equivalent to measuring all the qubits simultaneously. Furthermore, as long as all the qubits are measured, the sequence in which they are measured is also inconsequential. (These factors play are important to consider for real quantum computers with imperfect qubits, but we restrict the discussion to ideal systems in this tutorial.) 

In [15]:
operation WhichBasisState (qs : Qubit[]) : Int[] {
    let N = Length(qs);
    mutable array = new Int[N];
    for (i in 0 .. N - 1) {
        set array w/= i <- M(qs[i]) == One ? 1 | 0;
    }
    Message("Test passed");
    return array;
}

WhichBasisState

In [109]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Convert;
operation WhichBasisStateDemo () : Unit{
    using (qs = Qubit[3]) {
        X(qs[0]);
        H(qs[0]);
        H(qs[1]);
        X(qs[2]);
        H(qs[2]);
        
        Message("Qubits initialized to the state state |-+-❭ ");
    
        mutable SystemState = "";
        mutable qubitState = "";
        for (i in 0 .. Length(qs) - 1) {
            set qubitState = Measure([ PauliX], [qs[i]])  == One ? "-" | "+";
            // set array w/= i <- M(qs[i]) == One ? 1 | 0;
            set SystemState = SystemState + qubitState;
        }
        
        Message($"Qubits measured to be in the state |{SystemState}❭");
        ResetAll(qs);
    }
}

WhichBasisStateDemo

In [110]:
%simulate WhichBasisStateDemo

Qubits initialized to the state state |-+-❭ 
Qubits measured to be in the state |-+-❭


()

### <span style="color:blue">Demo: Measurement statistics</span>

Multi-qubit measurements can be implemented in `Q#` using the `Measure` operation. As outined in the [previous tutorial](addlink), using appropriate unitary rotations in conjuction with the Measure operation, it is possible to implement measurements in any orthogonal basis of one's choice. 

The following cell contains code demonstrating the use of the `Measure` operation. First, we prepare a 2-qubit system in the following state:
$$
\ket{\psi} = \frac{2/3}
$$

that the theoretical and experimental values of the probability outcomes indeed match with each other. We repeatedly prepare the same state $\ket \psi = 0.6 \ket 0 + 0.8 \ket 1$ and measure it in the computational basis $100$ times. At the end, we expect $0$ to be measured approximately $36$ times, and $1$ to be measured approximately $64$ times. Note that since measurements are probabilistic, we do not expect the results to match these values exactly.

To add:
- Entanglement
- Partial measurements and projection operators

### <span style="color:blue">Exercise 3</span>: Distinguish orthogonal states

**Input**: A 3-qubit system which is guaranteed to be in either the $\ket{++-}$ state, or the $\ket{---}$ state.

**Output**: `true` if the qubit is in the $\ket{++-}$ state, or `false` if it was in the $\ket{---}$ state. The state of the qubits at the end of the operation does not matter

In [2]:
// %kata T3_IsQubitMinus

operation DistinguishMultiQubitState (qs : Qubit[]) : Bool {
    return Measure([PauliX], qs[0]) == Zero;
}

G:\snippet_.qs(4,30): error QS6210: The type of the given argument does not match the expected type. Got an argument of type Qubit, expecting one of type Qubit[] instead.
